<a href="https://colab.research.google.com/github/Chris-Congleton/MSc-Thesis/blob/main/Analysis/2021_SourceClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports


In [ ]:
!pip install -U gensim
!pip install pyLDAvis
!pip install shap
!pip install datasets

import json, os
import pandas as pd
import numpy as np
import nltk
import glob
import matplotlib.pyplot as plt
import datetime
from tqdm import tqdm
nltk.download('punkt')

# regular expression library
import re
# wordcloud library
from wordcloud import WordCloud
import pprint

import gensim
from gensim.utils import simple_preprocess
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('dutch')

import gensim.corpora as corpora

import pyLDAvis.gensim_models as gensimvis
import pickle 
import pyLDAvis

#Visualisation
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction                      
import IPython

from gensim.parsing.preprocessing import preprocess_string
from gensim import utils
import gensim.models

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier

import shap
import datasets

     |████████████████████████████████| 24.1 MB 2.1 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
     |████████████████████████████████| 1.7 MB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=a9f15aff89c6b10a88acdfec9505fe17ba9d56e3b0d3c2601aab4fe87c566894
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis
     |████████████████████████████████| 564 kB 5.3 MB/s 
     |████████████████████████████████| 325 kB 5.2 MB/s 
     |████████████████████████████████| 212 kB 61.6 MB/s 
     |████████████████████████████████| 67 kB 5.4 MB/s 
     |████████████████████████████████| 13

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
/usr/local/lib/python3.7/dist-packages/numba/core/types/__init__.py:108: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` t

# Loading and cleaning data

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
!dir gdrive/MyDrive/CS/Thesis/Testset_2021/

HetParool_2021.json  LDA_HetParool.png	LDA_Trouw.png
LDA_all.html	     LDA__.html		LDA_Volkskrant.html
LDA_All.html	     LDA_NRC.html	NRC_2021.json
LDA_all.png	     LDA_NRC.png	Telegraaf_2021.json
LDA_All.png	     LDA__.png		Trouw_2021.json
LDA_HetParool.html   LDA_Trouw.html	Volkskrant_2021.json


In [ ]:
path_NRC = 'gdrive/MyDrive/CS/Thesis/Testset_2021/NRC_2021.json'
path_Trouw = 'gdrive/MyDrive/CS/Thesis/Testset_2021/Trouw_2021.json'
path_HetParool = 'gdrive/MyDrive/CS/Thesis/Testset_2021/HetParool_2021.json'
path_Telegraaf = 'gdrive/MyDrive/CS/Thesis/Testset_2021/Telegraaf_2021.json'
path_Volkskrant = 'gdrive/MyDrive/CS/Thesis/Testset_2021/Volkskrant_2021.json'

In [ ]:
df_NRC = pd.read_json(path_NRC)
df_Trouw = pd.read_json(path_Trouw)
df_HetParool = pd.read_json(path_HetParool)
df_Telegraaf = pd.read_json(path_Telegraaf)
df_Volkskrant = pd.read_json(path_Volkskrant)

In [ ]:
df_NRC["Source"] = "NRC"
df_Trouw["Source"] = "Trouw"
df_HetParool["Source"] = "HetParool"
df_Telegraaf["Source"] = "Telegraaf"
df_Volkskrant["Source"] = "Volkskrant"

In [ ]:
print(df_NRC.shape)
print(df_Trouw.shape)
print(df_HetParool.shape)
print(df_Volkskrant.shape)

(38334, 7)
(21077, 7)
(18551, 7)
(25773, 7)


## Concat data

In [ ]:

dfs = [df_NRC, df_Trouw, df_HetParool, df_Volkskrant]
df = pd.concat(dfs, ignore_index=True)

df.head()

,url,timestamp,title,publisherID,cleantext,category,Source
0,https://www.nrc.nl/nieuws/2021/01/31/een-herha...,2021-01-31 20:18:43,Een herhaling van vorig seizoen is zo goed als...,Fabian van der Poll,De eerste maand van 2021 is uitstekend verlope...,None,NRC
1,https://www.nrc.nl/nieuws/2021/01/31/avondklok...,2021-01-31 20:02:10,‘Avondklok waarschijnlijk niet verlengd als be...,Steven Musch,De avondklok wordt waarschijnlijk niet verleng...,None,NRC
2,https://www.nrc.nl/nieuws/2021/01/31/school-op...,2021-01-31 19:58:21,"School open? Dan testen, ventilatie en mondkapjes",Karel Berkhout,Door de heropening van de scholen loopt het aa...,None,NRC
3,https://www.nrc.nl/nieuws/2021/01/31/scholen-o...,2021-01-31 19:45:18,"De basisscholen gaan open, met een paar nieuwe...",Claudia Kammer,"Het werd alom verwacht, en zondag aan het ei...",None,NRC
4,https://www.nrc.nl/nieuws/2021/01/31/arrestati...,2021-01-31 19:43:49,Arrestatie Russische betoger,None,,None,NRC


## Remove missing text

In [ ]:
df = df[df['cleantext'].notna()]
print(df.shape)

(103723, 7)


# Source Classifier

## Preprocessing

In [ ]:
# Select only political articles

articles = df.drop(columns=['url', 'publisherID', 'category',], axis=1)
articles = articles[articles['cleantext'].notna()]
# Remove punctuation
articles['cleantext_processed'] = articles['cleantext'].map(lambda x: re.sub('[,\\.!?]', '', x))
# Save D66
articles['cleantext_processed'] = articles['cleantext_processed'].map(lambda x: re.sub('D66', 'dzeszes', x))
# Save BIJ1
articles['cleantext_processed'] = articles['cleantext_processed'].map(lambda x: re.sub('BIJ1', 'bijeeen', x))
# Save 50PLUS
articles['cleantext_processed'] = articles['cleantext_processed'].map(lambda x: re.sub('50PLUS', 'vijfplus', x))
# Save JA21
articles['cleantext_processed'] = articles['cleantext_processed'].map(lambda x: re.sub('JA21', 'jatweeeen', x))
# Convert the titles to lowercase
articles['cleantext_processed'] = articles['cleantext_processed'].map(lambda x: x.lower())

p_and_p =  ['vvd', 'dzeszes', 'pvv', 'cda', 'pvda', 'sp', 'gl', 'fvd', 'pvdd',
            'cu', 'sgp', 'volt', 'jatweeeen', 'bbb', 'bijeeen', 'vijfplus', 'rutte', 'kaag',
            'wilders', 'hoekstra', 'ploumen', 'marijnissen', 'klaver', 'baudet',
            'ouwehand', 'segers', 'staaij', 'dassen', 'eerdmans', 'plas',
            'simons', 'den haan']

pat = '|'.join(r"\b{}\b".format(x) for x in p_and_p)
articles = articles[articles['cleantext_processed'].str.contains(pat)]


In [ ]:
# Remove stopwords

stop_words.extend(['nbsp', 'rsquo', 'lsquo','euml', 'rdquo', 'quot', 'ldquo',
                    'we', 'wel', 'jaar', 'zegt', 'gaat','moeten', 'volgens',
                    'heel', 'maken', 'weer', 'gaan', 'twee', 'mensen', 'waar', 'alleen',
                    'komt', 'zoals', 'alle', 'wij', 'zei','komen','tussen', 'eerste','zien'
                    'eigen', 'leven','grote', 'nieuwe','mee','goed', 'aantal','laten'
                    ,'tweede', 'af', 'eerder', 'eigen', 'elkaar','afgelopen', 'week',
                    'zeggen', 'echt', 'dag', 'zien', 'steeds', 'laten', 'willen', 'waarin',
                   'duizend','voormalig', 'nu', 'we'])

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
            if word not in stop_words] for doc in texts]

articles['cleantext_processed'] = remove_stopwords(articles['cleantext_processed'])

articles['cleantext_processed'] = [' '.join(map(str, l)) for l in articles['cleantext_processed']]

In [ ]:
print(articles['cleantext_processed'].head())

1     avondklok waarschijnlijk verlengd februari zol...
3     alom verwacht zondag eind bevestigden demissio...
6     wantrouwen overheid spuit eruit eindelijk ligt...
15    lang gedacht stijging zeespiegel einde eeuw me...
22    basisscholen kinderopvang vanaf maandag februa...
Name: cleantext_processed, dtype: object


In [ ]:
Xy = articles.drop(columns=['timestamp', 'title'])
X = Xy['cleantext_processed']
y = Xy['Source']

In [ ]:
print(X.shape)

(15498,)


In [ ]:
print(y.shape)

(15498,)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7)
X = tfidfconverter.fit_transform(X).toarray()

# Decision Tree Classifier

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_encoded_input, y, test_size=0.2, random_state=0)

ValueError: ignored

In [ ]:

classifier = DecisionTreeClassifier(max_depth=20,random_state=17)
classifier.fit(X_train, y_train) 

y_pred = classifier.predict(X_test)



ValueError: ignored

In [ ]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

NameError: ignored

In [ ]:
features = list(tfidfconverter.get_feature_names_out())


In [ ]:
from sklearn import tree
text_representation = tree.export_text(classifier, feature_names=features)
print(text_representation)

|--- demissionair <= 0.00
|   |--- nrc <= 0.01
|   |   |--- amsterdam <= 0.04
|   |   |   |--- gewoon <= 0.04
|   |   |   |   |--- persbureau <= 0.01
|   |   |   |   |   |--- gisteren <= 0.05
|   |   |   |   |   |   |--- krant <= 0.02
|   |   |   |   |   |   |   |--- slechts <= 0.04
|   |   |   |   |   |   |   |   |--- onze <= 0.03
|   |   |   |   |   |   |   |   |   |--- inmiddels <= 0.03
|   |   |   |   |   |   |   |   |   |   |--- zitten <= 0.03
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 10
|   |   |   |   |   |   |   |   |   |   |--- zitten >  0.03
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 10
|   |   |   |   |   |   |   |   |   |--- inmiddels >  0.03
|   |   |   |   |   |   |   |   |   |   |--- hadden <= 0.04
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 10
|   |   |   |   |   |   |   |   |   |   |--- hadden >  0.04
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth

# SVM

In [ ]:
from sklearn import svm

In [ ]:
clf = svm.SVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[ 118  232   45  105]
 [  26 1019   62  165]
 [  21  286  135  124]
 [  29  304   60  369]]
              precision    recall  f1-score   support

   HetParool       0.61      0.24      0.34       500
         NRC       0.55      0.80      0.65      1272
       Trouw       0.45      0.24      0.31       566
  Volkskrant       0.48      0.48      0.48       762

    accuracy                           0.53      3100
   macro avg       0.52      0.44      0.45      3100
weighted avg       0.53      0.53      0.50      3100

0.5293548387096774


# xgboost


In [ ]:
import xgboost
model = xgboost.XGBClassifier().fit(X_train, y_train)
y_pred = model.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[ 114  270   23   93]
 [  29 1079   26  138]
 [  31  356   59  120]
 [  39  407   27  289]]
              precision    recall  f1-score   support

   HetParool       0.54      0.23      0.32       500
         NRC       0.51      0.85      0.64      1272
       Trouw       0.44      0.10      0.17       566
  Volkskrant       0.45      0.38      0.41       762

    accuracy                           0.50      3100
   macro avg       0.48      0.39      0.38      3100
weighted avg       0.49      0.50      0.45      3100

0.49709677419354836


# RobBert

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 4.9 MB/s 
     |████████████████████████████████| 596 kB 42.5 MB/s 
     |████████████████████████████████| 895 kB 36.9 MB/s 
     |████████████████████████████████| 6.5 MB 42.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
tokenizer = RobertaTokenizer.from_pretrained("pdelobelle/robbert-v2-dutch-base")

Downloading:   0%|          | 0.00/715k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/374k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/660 [00:00<?, ?B/s]

In [ ]:
articles = df.drop(columns=['url', 'publisherID', 'category','timestamp', 'title'], axis=1)
articles = articles[articles['cleantext'].str.len()>5]
df_bert = articles

In [ ]:
df_bert.head()

,cleantext,Source
0,De eerste maand van 2021 is uitstekend verlope...,NRC
1,De avondklok wordt waarschijnlijk niet verleng...,NRC
2,Door de heropening van de scholen loopt het aa...,NRC
3,"Het werd alom verwacht, en zondag aan het ei...",NRC
5,"Het was lang wachten, maar 36 jaar nadat Jan T...",NRC


In [ ]:
def preprocess_function(examples):
    return tokenizer(examples['cleantext'], truncation=True, padding=True, max_length=128)

In [ ]:
Xy_dd = datasets.Dataset.from_pandas(df_bert)

In [ ]:
Xy_dd = Xy_dd.train_test_split(test_size=0.2)

In [ ]:
tokenized = Xy_dd.map(preprocess_function, batched=True)

  0%|          | 0/78 [00:00<?, ?ba/s]

  0%|          | 0/20 [00:00<?, ?ba/s]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
model = RobertaForSequenceClassification.from_pretrained("pdelobelle/robbert-v2-dutch-base", num_labels=4)

Downloading:   0%|          | 0.00/446M [00:00<?, ?B/s]

Some weights of the model checkpoint at pdelobelle/robbert-v2-dutch-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at pdelobelle/robbert-v2-dutch-base and are newly initialized: ['classifier.out_proj.bias', 'clas

In [ ]:
training_args = TrainingArguments(

    output_dir="./results",

    learning_rate=2e-5,

    per_device_train_batch_size=16,

    per_device_eval_batch_size=16,

    num_train_epochs=5,

    weight_decay=0.01,

)

trainer = Trainer(

    model=model,

    args=training_args,

    train_dataset=tokenized['train'],

    eval_dataset=tokenized['test'],

    tokenizer=tokenizer,

    data_collator=data_collator,

)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: cleantext, __index_level_0__, Source. If cleantext, __index_level_0__, Source are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
***** Running training *****
  Num examples = 77472
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 24210


KeyError: ignored

In [ ]:
tf_train_dataset = tokenized["train"].to_tf_dataset(

    columns=["attention_mask", "input_ids", "Source"],

    shuffle=True,

    batch_size=16,

    collate_fn=data_collator,

)

tf_validation_dataset = tokenized["train"].to_tf_dataset(

    columns=["attention_mask", "input_ids", "Source"],

    shuffle=False,

    batch_size=16,

    collate_fn=data_collator,

)

In [ ]:
from transformers import create_optimizer

import tensorflow as tf

batch_size = 16

num_epochs = 5

batches_per_epoch = len(tokenized["train"]) // batch_size

total_train_steps = int(batches_per_epoch * num_epochs)

optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [ ]:
import tensorflow as tf

model.compile(optimizer=optimizer)

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3)